<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/signature_verification_vggnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Keras libraries and packages
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 128, 128



In [8]:
# Updating Keras
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [10]:
#Load the pretrained Network
vgg16_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height,img_width,3), pooling=None, classes=1000)
print("pretrained Network is loaded")

58892288/58889256 [==============================] - 2s 0us/step
pretrained Network is loaded


In [11]:
# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")
    

Pretrained layers are freezed


In [12]:
# Model Buliding
model = Sequential()
model.add(vgg16_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(1, activation='sigmoid', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [14]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset1/training_set'
val_data_dir ='/content/drive/My Drive/Dataset1/testing_set'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_epochs = 35
print("Input parameters are assigned")

Input parameters are assigned


In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False,class_mode='binary')


Found 2112 images belonging to 2 classes.
Found 544 images belonging to 2 classes.


In [22]:
import os
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
folder = '/content/drive/My Drive/Dataset1/training_set/'
photos, labels = list(), list()
for r, d, f in os.walk(folder):
    for file in f:
        if ".png" in file:
          if(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/training_set/genuine/{}'.format(file)):
            output=1
          elif(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/training_set/forged/{}'.format(file)):
            output=0
        labels.append(output)
labels_train=np.array(labels)
print(os.path.join(r, file))
print(labels_train.shape)
labels_train=np.expand_dims(labels_train, axis=1)
print(labels_train)
print(labels_train.shape)

/content/drive/My Drive/Dataset1/training_set/genuine/006_01.PNG
(2112,)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
(2112, 1)


In [25]:
import os
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
folder = '/content/drive/My Drive/Dataset1/testing_set/'
photos, labels = list(), list()
for r, d, f in os.walk(folder):
    for file in f:
        if ".png" in file:
          if(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/testing_set/genuine/{}'.format(file)):
            output=1
          elif(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/testing_set/forged/{}'.format(file)):
            output=0
        labels.append(output)
labels_test=np.array(labels)
print(os.path.join(r, file))
print(labels_test.shape)
labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)

/content/drive/My Drive/Dataset1/testing_set/genuine/NFI-01401014.png
(544,)
(544, 1)


In [0]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, nb_epoch=nb_epochs, validation_data=validation_generator)

print('Training Completed!')


Epoch 1/35


KeyboardInterrupt: ignored

In [30]:
 # save model and architecture to single file
model.save('/content/drive/My Drive/Colab Notebooks/Signature Verification VGGnet Model/model_vggnet_signature_verification.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               1048704   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 15,771,713
Trainable params: 1,057,025
Non-trainable params: 14,714,688
_________________________________________________________________
Saved model to disk


In [33]:
# save weights to single file
model.save_weights('/content/drive/My Drive/Colab Notebooks/Signature Verification VGGnet Model/model_vggnet_weights_signature_verification.h5')
print("Weights saved in Drive")

Weights saved in Drive


In [36]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Colab Notebooks/Signature Verification VGGnet Model/model_vggnet_signature_verification.h5')
print("Model is Loaded")


Model is Loaded


In [39]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [40]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [41]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Colab Notebooks/Signature Verification VGGnet Model/intermediate_model_vggnet_signature_verification.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16_input (InputLayer)     (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               1048704   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 15,771,648
Trainable params: 1,056,960
Non-trainable params: 14,714,688
_________________________________________________________________
Saved Intermediate model to disk


In [43]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Colab Notebooks/Signature Verification VGGnet Model/intermediate_model_vggnet_signature_verification.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [44]:
intermediate_output = model.predict(train_generator)
X_train = np.array(intermediate_output)
print(X_train)
print(X_train.shape)

[[1.0880522  0.7292641  0.         ... 0.         0.         0.        ]
 [0.6750734  0.7107771  0.         ... 0.         0.         0.        ]
 [0.38623777 0.7267865  0.         ... 0.         0.         0.        ]
 ...
 [0.816037   0.52971834 0.         ... 0.         0.         0.        ]
 [0.76509434 0.80243886 0.         ... 0.         0.         0.        ]
 [0.71141696 0.63068485 0.         ... 0.         0.         0.        ]]
(2112, 64)


In [45]:
intermediate_output_test = model.predict(validation_generator)
X_test = np.array(intermediate_output_test)
print(X_test)
print(X_test.shape)

[[0.4430195  0.75575554 0.         ... 0.13835432 0.         0.        ]
 [0.6744821  0.5393736  0.         ... 0.2250725  0.         0.        ]
 [0.33553204 0.68687344 0.         ... 0.         0.         0.        ]
 ...
 [0.5958545  0.7740291  0.         ... 0.         0.         0.        ]
 [0.59099334 0.6355299  0.         ... 0.         0.         0.        ]
 [0.782453   0.47949722 0.         ... 0.         0.         0.        ]]
(544, 64)


In [46]:
# Training Label feature identification(y_train)
 import numpy as np
 batch_size=32
 sample_count=2112
 features = np.zeros(shape=(2112, 64))  # Must be equal to the output of the convolutional base
 labels = np.zeros(shape=(2112))
 i = 0
 for inputs_batch, labels_batch in train_generator:
   features_batch = model.predict(inputs_batch)
   features[i * batch_size: (i + 1) * batch_size] = features_batch
   labels[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels.shape)

(2112,)


In [48]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train.shape)
print(labels_train)

(2112, 64)
(2112, 1)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [49]:
# identification of test labels
 import numpy as np
 batch_size=32
 sample_count=544
 features_test = np.zeros(shape=(544, 64))  # Must be equal to the output of the convolutional base
 labels_test = np.zeros(shape=(544))
 i = 0
 for inputs_batch, labels_batch in validation_generator:
   features_batch = model.predict(inputs_batch)
   features_test[i * batch_size: (i + 1) * batch_size] = features_batch
   labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels_test.shape)
 print(features_test.shape)
 print(labels_test)

(544,)
(544, 64)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [50]:
# #identification of testing Labels
print(features_test.shape)
labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)

(544, 64)
(544, 1)


In [51]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train,labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[144 124]
 [ 99 177]]
0.5900735294117647


In [53]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[180  88]
 [ 89 187]]
0.6746323529411765


In [54]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))


[[173  95]
 [ 97 179]]
0.6470588235294118


In [55]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[152 116]
 [117 159]]
0.5716911764705882


In [56]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[ 96 172]
 [ 40 236]]
0.6102941176470589


In [57]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[135 133]
 [ 41 235]]
0.6801470588235294


In [58]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[147 121]
 [ 98 178]]
0.5974264705882353


In [60]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Colab Notebooks/Signature Verification VGGnet Model/model_vggnet_signature_verification.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
17/17 [==============================] - 92s 5s/step
test loss, test acc: [0.45949941873550415, 0.5091911554336548]
